# IMPORT THE LIBRARIES

In [1]:
#exc1
import pandas as pd
import numpy as np

from dateutil.parser import parse

from datetime import date
import datetime



#Sheet From Pi
#Step1: Load the Data for the month
df_Pi = pd.read_excel(r'C:\Users\v.t.flores\Documents\Pressure Filter Inv_2020_PF_stagecheck.xlsx', sheet_name='PiData_PressureFilter', index_col=False)

#Drop the row[1]
df_Pi = df_Pi.drop(df_Pi.index[0])

#Step2: Parse the DATETIME column
df_Pi['DATETIME'] = df_Pi['DATETIME'].apply(lambda x: parse(str(x)))

df_Pi.set_index('DATETIME', inplace=True)
df_comb = df_Pi


def getTheLocalMinMaxRev(df_col1, df_col2):
    df_col1 = pd.to_numeric(df_col1, errors='coerce')
    b = (np.diff(np.sign(np.diff(df_col1))) > 0).nonzero()[0] + 1         # local min
    #Variable b contains the list of indices for where there are minimum values detected from the rows
    
    df_col2 = pd.to_numeric(df_col2, errors='coerce')
    f = (np.diff(np.sign(np.diff(df_col2))) < 0).nonzero()[0] + 1         # local max
    
    #Variable f contains the list of indices for where there are maximum values detected from the rows
    
    try:
        ft_cyc_lim = {}
        for i in range(0, len(b)):
            for j in range(0, len(f)):
                if b[i] < f[j]:
                    cyc_start = b[i]
                    cyc_end = f[j]
                    ft_cyc_lim.setdefault(cyc_start,cyc_end) #The ft_cyc_lim is the dictionary of the local min, max pair
                    pass
    except IndexError:
        print('IndexError')
    return ft_cyc_lim


def generate_DF(df, str_colname, dictParameter):
    # Prepare the df_coltitle as df_comb['filtration']
    #-------------str_colname as 'A_t_feed'
    #-------------dict_FT as the dictionary containing the local min and local max filtration time indices
    #---the key is the local mininmum filtration time
    #---the key element is the local maximum filtration time which signifies the end of the filtration time
    
    A_Filt_MaxList= list()
    A_Filt_MaxList[:] = []
    A_feedt_Val_List = list()
    A_feedt_Val_List[:] = []
    A_feedt_index_List = list()
    A_feedt_index_List[:] = []
    A_Pair_Filt_List = list()
    A_Pair_Filt_List[:] = []
    try:
        for i in dictParameter.keys():
            A_feedt = df[str_colname][dictParameter[i]]
            A_feedt_Val_List.append(A_feedt)
            A_feedt_index_List.append(df.index[dictParameter[i]])
        A_Pair_Filt_List = list(zip(A_feedt_Val_List, A_feedt_index_List[0:]))
    except:
        pass
    df_A_Pair_Filt_t = pd.DataFrame(A_Pair_Filt_List)
    
    df_A_Pair_Filt_t.columns = [str_colname, 'DATETIME']
    df_A_Pair_Filt_t.set_index('DATETIME', inplace=True)  
    return df_A_Pair_Filt_t
#--------------------------------------------

#create a dictionary of the local minimum and local maximum values for each parameter
#Use the function getTheLocalMinMaxRev()

dict_AT =  getTheLocalMinMaxRev(df_comb['A_t_FEED'], df_comb['A_t_FEED'])
dict_N2_AT = getTheLocalMinMaxRev(df_comb['A_t_DRY'], df_comb['A_t_DRY'])
dict_CWSH1_AT = getTheLocalMinMaxRev(df_comb['A_t_CWSH1'], df_comb['A_t_CWSH1'])
dict_CWSH2_AT = getTheLocalMinMaxRev(df_comb['A_t_CWSH2'], df_comb['A_t_CWSH2'])
dict_P1_AT = getTheLocalMinMaxRev(df_comb['A_t_PRESS1'], df_comb['A_t_PRESS1'])
dict_P2_AT = getTheLocalMinMaxRev(df_comb['A_t_PRESS2'], df_comb['A_t_PRESS2'])


dict_BT =  getTheLocalMinMaxRev(df_comb['B_t_FEED'], df_comb['B_t_FEED'])
dict_N2_BT = getTheLocalMinMaxRev(df_comb['B_t_DRY'], df_comb['B_t_DRY'])
dict_CWSH1_BT = getTheLocalMinMaxRev(df_comb['B_t_CWSH1'], df_comb['B_t_CWSH1'])
dict_CWSH2_BT = getTheLocalMinMaxRev(df_comb['B_t_CWSH2'], df_comb['B_t_CWSH2'])
dict_P1_BT = getTheLocalMinMaxRev(df_comb['B_t_PRESS1'], df_comb['B_t_PRESS1'])
dict_P2_BT = getTheLocalMinMaxRev(df_comb['B_t_PRESS2'], df_comb['B_t_PRESS2'])

#Create a dataframe from the actual parameter values using the dictionary created previously

df_FT_feed_t =  generate_DF(df_comb, 'A_t_FEED', dict_AT)
df_FT_N2_t =  generate_DF(df_comb, 'A_t_DRY', dict_N2_AT)
df_FT_CWSH1_t =  generate_DF(df_comb, 'A_t_CWSH1', dict_CWSH1_AT)
df_FT_CWSH2_t =  generate_DF(df_comb, 'A_t_CWSH2', dict_CWSH2_AT)
df_FT_P1_t =  generate_DF(df_comb, 'A_t_PRESS1', dict_P1_AT)
df_FT_P2_t =  generate_DF(df_comb, 'A_t_PRESS2', dict_P2_AT)

#Generate the dataframes for A
df_A_N2DRY = generate_DF(df_comb, 'A_t_DRY', dict_AT)
df_A_t_FEED = generate_DF(df_comb, 'A_t_FEED', dict_N2_AT)
df_A_t_CWSH1 = generate_DF(df_comb, 'A_t_CWSH1', dict_CWSH1_AT)
df_A_t_CWSH2 = generate_DF(df_comb, 'A_t_CWSH2', dict_CWSH2_AT)
df_A_t_PRESS1 = generate_DF(df_comb, 'A_t_PRESS1', dict_P1_AT)
df_A_t_PRESS2 = generate_DF(df_comb, 'A_t_PRESS2', dict_P2_AT)

#Generate the dataframes for B
df_B_N2DRY = generate_DF(df_comb, 'B_t_DRY', dict_BT)
df_B_t_FEED = generate_DF(df_comb, 'B_t_FEED', dict_N2_BT)
df_B_t_CWSH1 = generate_DF(df_comb, 'B_t_CWSH1', dict_CWSH1_BT)
df_B_t_CWSH2 = generate_DF(df_comb, 'B_t_CWSH2', dict_CWSH2_BT)
df_B_t_PRESS1 = generate_DF(df_comb, 'B_t_PRESS1', dict_P1_BT)
df_B_t_PRESS2 = generate_DF(df_comb, 'B_t_PRESS2', dict_P2_BT)


df1A = df_A_N2DRY.join(df_A_t_FEED, how='outer')
df2A = df_A_t_CWSH1.join(df_A_t_CWSH2, how='outer')
df3A = df_A_t_PRESS1.join(df_A_t_PRESS2, how='outer')
df4A = df1A.join(df2A, how='outer')
df5A = df3A.join(df4A, how='outer')

df1B = df_B_N2DRY.join(df_B_t_FEED, how='outer')
df2B = df_B_t_CWSH1.join(df_B_t_CWSH2, how='outer')
df3B = df_B_t_PRESS1.join(df_B_t_PRESS2, how='outer')
df4B = df1B.join(df2B, how='outer')
df5B = df3B.join(df4B, how='outer')



In [2]:
df5B

,B_t_PRESS1,B_t_PRESS2,B_t_DRY,B_t_FEED,B_t_CWSH1,B_t_CWSH2
DATETIME,,,,,,
2020-07-01 00:03:00,100.0,NaN,NaN,NaN,NaN,NaN
2020-07-01 00:06:00,NaN,NaN,NaN,NaN,NaN,160.0
2020-07-01 00:10:00,NaN,120.0,NaN,NaN,NaN,NaN
2020-07-01 00:13:00,NaN,NaN,NaN,316.0,NaN,NaN
2020-07-01 00:21:00,NaN,NaN,110.0,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-07-28 12:13:00,NaN,NaN,NaN,NaN,110.0,NaN
2020-07-28 12:16:00,110.0,NaN,NaN,NaN,NaN,NaN
2020-07-28 12:19:00,NaN,NaN,NaN,NaN,NaN,130.0
